In [2]:
# % rm -r distributed-learning/
! git clone --recurse-submodules https://github.com/ekiuled/distributed-learning
# ! git submodule init
# ! git submodule update
% cd distributed-learning
! git checkout convergence

Cloning into 'distributed-learning'...
remote: Enumerating objects: 342, done.
remote: Total 342 (delta 0), reused 0 (delta 0), pack-reused 342
Receiving objects: 100% (342/342), 23.59 MiB | 20.54 MiB/s, done.
Resolving deltas: 100% (185/185), done.
Submodule 'wide-resnet.pytorch' (https://github.com/meliketoy/wide-resnet.pytorch.git) registered for path 'wide_resnet_submodule'
Cloning into '/content/distributed-learning/wide_resnet_submodule'...
remote: Enumerating objects: 13, done.        
remote: Counting objects: 100% (13/13), done.        
remote: Compressing objects: 100% (10/10), done.        
remote: Total 124 (delta 4), reused 11 (delta 3), pack-reused 111        
Receiving objects: 100% (124/124), 674.93 KiB | 11.64 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Submodule path 'wide_resnet_submodule': checked out '292b3ede0651e349dd566f9c23408aa572f1bd92'
/content/distributed-learning
Branch 'convergence' set up to track remote branch 'convergence' from 'origin'.
Switche

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from pprint import pprint

from utils.consensus_node import ConsensusNode
from utils.master_node import MasterNode

import wide_resnet_submodule.config as cf
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim

from wide_resnet_submodule.networks import *
from utils.graphs_config import ABC_3, TOP_5, LONELY, edges2topology, adj2edges
from utils.functions import fit_batch_cifar, update_params_cifar, calc_accuracy_cifar, get_cumulative_train_loss